### Import the text representation from local module

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

### Import the necessary dependencies to extract the text from the pdf

In [2]:
import numpy as np
import pdfparser.poppler as pdf
from simple_text_representation.classes import Text, Paragraph, Sentence, Word
from simple_text_representation.models import Database, TextModel, ParagraphModel, SentenceModel
from dateutil.parser import parse

### Initialize the Database connection

In [3]:
database = Database('educationalTexts', 'postgres', '', '0.0.0.0', 5432)

''' If the database it's empty, use the following command to create all the tables '''
database.createTables([TextModel, ParagraphModel, SentenceModel])

### Set global variables that describe the structure of the pdf file

In [4]:
INITIAL_PAGE = 5
PAGES_PER_TEXT = 2
ALLOWED_TAGS = ['APRENDEMOS', 'PRACTICAMOS', 'EXTENSIÓN']

In [5]:
def convertPDFtoText(fileName):
    document = pdf.Document(fileName.encode('UTF-8'))

    print('No of pages', document.no_of_pages)
#     for page in document:
#         print('Page', page.page_no, 'size =', page.size)
#         for f in page:
# #             print(' '*1,'Flow')
#             for b in f:
# #                 print(' '*2,'Block', 'bbox=', b.bbox.as_tuple())
#                 for l in b:
#                     print(' '*3, l.text)
#                     #assert l.char_fonts.comp_ratio < 1.0
# #                     for i in range(len(l.text)):
# #                         print(l.text[i].encode('UTF-8'), '(%0.2f, %0.2f, %0.2f, %0.2f)'% l.char_bboxes[i].as_tuple(),\
# #                             l.char_fonts[i].name, l.char_fonts[i].size, l.char_fonts[i].color,)
#                     print()
#                 print("=============================================================================")
    return document

In [6]:
document = convertPDFtoText("../files/comprension_lectora_1.pdf")

No of pages 194


# Limpieza de Textos

## Estructura actual de un Texto

Despues de realizar la estracción de textos con la herramienta PDFtoText se consigue un listado de elementos, donde cada elemento representa una parte de un texto y cada par de elementos adyacentes representan un texto completo. En su estado original, estos traen mucha información innecesaria que debe ser removida y que detallará a continuación.

* Tema a tratar
* Etiqueta de la sección
* Indicaciones del texto
* Titulo del texto
* Referencias de imagenes
* Preguntas relacionas al texto.
* Referencias del texto
* Pie de pagina
* Descripción de la hoja actual.


## Retos de la Limpieza

* Conservar la estructura original del texto, es decir, que todos los parrafos sean identificables.
* Eliminar todas las referencias de imagenes existentes, ya que se usan de forma indeterminada en distintas partes de un texto.
* Adaptar las diferentes estructuras de los archivos PDF provistos

## Pasos de Limpieza

* Usar la Etiqueta de cada texto para determinar el inicio de este.
    * Remover la información previa a la etiqueta
    * Remover la información antes del inicio del texto y despues de la etiqueta
* Eliminar la información extra por hoja.
    * Eliminar la información extra de la primera pagina (referencias y pie de pagina)
    * Eliminar la información extra de la segunda pagina (referencias, preguntas y pie de pagina)
* Eliminar las referencias a imagenes aún existentes.

## Resultado esperado

Finalmente, se espera obtener un arreglo de elementos donde cada represente a un texto completo y que este listo para ser pre-procesado.

In [7]:
def hasAllowedTag(line):
    return any(TAG in line for TAG in ALLOWED_TAGS)

In [8]:
def transformToPython(page):
    listOfFlows = list()
    for flow in page:
        listOfBlocks = list()
        for paragraph in flow:
            listOfLines = ''
            for line in paragraph:
                listOfLines = joinLine(listOfLines, line.text)
#                 listOfLines.append(line.text)
#                 print(line.text)
#                 print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            listOfBlocks.append(listOfLines)
#             listOfBlocks.append(''.join(listOfLines))
        listOfFlows.append(listOfBlocks)

    return listOfFlows

In [9]:
def joinLine(listOfLines, line):
    newListOfLines = listOfLines
    separator = ''
    
    if (newListOfLines):
        if (newListOfLines[- 1] == '-'):
            newListOfLines = newListOfLines[:-1]
        else:
            separator = ' '
        newListOfLines = newListOfLines + separator + line
    else:
        newListOfLines = newListOfLines + line

    return newListOfLines

In [10]:
def cleanDocument(file):
    nextPageAdded = -1
    listOfTexts = list()
    text = list()
    
    for index,page in enumerate(file):
        if (index > INITIAL_PAGE):
            if (nextPageAdded == 1):
                text.append(transformToPython(page))
                textTuple = (text[0], text[1])
                listOfTexts.append(textTuple)
                nextPageAdded = -1
            else:
                text = list()
                listOfFlows = list()
                for flow in page:
                    listOfBlocks = list()
                    for paragraph in flow:
                        listOfLines = ''
                        for line in paragraph:
                            listOfLines = joinLine(listOfLines, line.text)
#                             print(line.text)
#                             print('========================================')
#                             listOfLines.append(line.text)
                            if (hasAllowedTag(line.text)):
                                nextPageAdded = 1
#                             print(line.text)
#                         listOfBlocks.append(''.join(listOfLines))
                        listOfBlocks.append(listOfLines)
#                         print("------------------------------------------------------")
                    listOfFlows.append(listOfBlocks)
                text.append(listOfFlows)

    return listOfTexts

### List of unformated Texts

In [11]:
listOfTexts = cleanDocument(document)

### Clean conditions

In [12]:
def hasLinkReference(block):
    return 'http' in block

In [13]:
def isBibliographicCitation(block):
    return 'Tomado de' in block

In [14]:
def isPageNumeration(block):
    try:
        int(block)
        return True
    except ValueError:
        return False

In [15]:
def onlyOneCharacter(block):
    return len(block) == 1

In [16]:
def isOnlyDate(block):
    try: 
        parse(block)
        return True
    except ValueError:
        return False

In [17]:
def isStartOfQuestionary(block):
    return ('Responde las preguntas' in block) or ('A partir de la lectura anterior' in block) or ('responde las preguntas' in block) or ('Utiliza el' in block)

In [18]:
def hasIndications(block):
    return 'Lee el siguiente text' in block

In [19]:
def specialCases(block):
    return ('Libro Comunicacion' in block) or ('Comprensión lectora' in block) or ('grado de secundaria' in block)

In [20]:
def cleanContiditions(block):
    return not ( hasLinkReference(block) or onlyOneCharacter(block) or isOnlyDate(block) or specialCases(block) or isPageNumeration(block) or isBibliographicCitation(block) )

In [21]:
def getSecondPage(newPage):
    secondPage = list()

    for block in newPage:
        for line in block:
            secondPage.append(line)
    
    return secondPage

In [22]:
def getTitleAndFirstPage(newPage):
    title = ''
    firstPage = list()

    for i,block in enumerate(newPage):
        for j,line in enumerate(block):
            if (i == 0 and j ==0):
                title = line
            else:
                firstPage.append(line)

    return title, firstPage

In [23]:
def cleanPage(page, isQuestionaryFound = False, isFirstPage = False):
    newPage = list()
    isTagFound = False
    isIndications = False
    questionary = isQuestionaryFound

    for flow in page:
        listOfBlocks = list()
        for block in flow:
            if (cleanContiditions(block)):
                listOfBlocks.append(block)
            isTagFound = isTagFound or hasAllowedTag(block)
            isIndications = isIndications or hasIndications(block)
            questionary = questionary or isStartOfQuestionary(block)
#         print(isTagFound, isIndications, questionary, listOfBlocks)
#         print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
        if questionary:
            break;
        if isTagFound or isIndications:
            newPage = list()
            listOfBlocks = list()
            isTagFound = False
            isIndications = False
        if (len(listOfBlocks) > 0):
            newPage.append(listOfBlocks)

#     print (listOfBlocks)
#     print('=================================================================')
    if (newPage and (isFirstPage and len(newPage) > 1)):
#         print(newPage)
#         print('========================================================')
        title, firstPage = getTitleAndFirstPage(newPage)
        return title, firstPage, questionary
    elif (newPage and not isFirstPage):
        return getSecondPage(newPage), questionary
    elif (not newPage and not isFirstPage):
        return [], False
    else:
        return '', [], False

### After declaring all the methods needed to clean the text, we do a simple text with the first two pages

In [24]:
len(listOfTexts)

60

In [25]:
testTextIndex = 0
mockQuestionary = False
testFirstPage = listOfTexts[testTextIndex][0]
testSecondPage = listOfTexts[testTextIndex][1]

In [26]:
testFirstPage

[['1.er grado de secundaria'],
 ['Ficha: Reconocemos la estructura del texto argumentativo', 'APRENDEMOS'],
 ['Lee el siguiente texto considerando las orientaciones que brinda tu docente.'],
 ['El acto solidario de la donación de órganos1'],
 ['Fuente de imagen: <https://goo.gl/exLu3M>'],
 ['Si bien los trasplantes se han convertido en una práctica habitual, aún persisten fuertes temores en la población para donar órganos, lograr su superación es la clave para aumentar el número de los donadores solidarios que hacen falta para salvar miles de vidas.',
  'Es preciso, entonces, que se aclaren algunas dudas para que las personas pierdan el miedo a donar. Primero, que lo complicado de los procedimientos de extirpación y trasplantación, en el que intervienen varios equipos médicos altamente especializados, vuelve muy difícil la existencia de mafias. Segundo, que la necesaria compatibilidad (afinidad de grupo sanguíneo) entre donante y receptor dificulta la posibilidad de muertes “a pedido”.

In [27]:
mockTitle, mockFirstPage, mockQuestionary = cleanPage(testFirstPage, mockQuestionary, True)
print(mockQuestionary, mockTitle, mockFirstPage)

False El acto solidario de la donación de órganos1 ['Si bien los trasplantes se han convertido en una práctica habitual, aún persisten fuertes temores en la población para donar órganos, lograr su superación es la clave para aumentar el número de los donadores solidarios que hacen falta para salvar miles de vidas.', 'Es preciso, entonces, que se aclaren algunas dudas para que las personas pierdan el miedo a donar. Primero, que lo complicado de los procedimientos de extirpación y trasplantación, en el que intervienen varios equipos médicos altamente especializados, vuelve muy difícil la existencia de mafias. Segundo, que la necesaria compatibilidad (afinidad de grupo sanguíneo) entre donante y receptor dificulta la posibilidad de muertes “a pedido”.']


In [28]:
testSecondPage

[['Comprensión lectora 1'],
 ['Fuente de imagen: <https://goo.gl/jPn0vJ>'],
 ['La última cuestión es la más compleja; en la actualidad, aunque alguien haya manifestado expresamente su voluntad de donar, es a la familia a la que se consulta en el momento en que la donación puede efectuarse. Como se entiende, tal consulta llega en un momento difícil y poco propicio para las reflexiones profundas, más aún si se tiene que tomar una decisión rápida.',
  'Por lo tanto, las campañas públicas deben esclarecer la naturaleza de los procedimientos técnicos y legales, para disipar miedos; pero, esencialmente, deben apuntar a que se tome conciencia de lo que significa salvar otra vida, porque para decidirlo en un momento crucial es necesario que la idea se haya considerado y discutido previamente, con reflexión y calma.'],
 ['6'],
 ['Interiores Libro Comunicacion 1 Secundaria Comprension Lectora .indd 6'],
 ['4/22/17 9:57 AM']]

In [29]:
mockSecondPage, mockQuestionary = cleanPage(testSecondPage, mockQuestionary)
print(mockQuestionary, mockSecondPage)

False ['La última cuestión es la más compleja; en la actualidad, aunque alguien haya manifestado expresamente su voluntad de donar, es a la familia a la que se consulta en el momento en que la donación puede efectuarse. Como se entiende, tal consulta llega en un momento difícil y poco propicio para las reflexiones profundas, más aún si se tiene que tomar una decisión rápida.', 'Por lo tanto, las campañas públicas deben esclarecer la naturaleza de los procedimientos técnicos y legales, para disipar miedos; pero, esencialmente, deben apuntar a que se tome conciencia de lo que significa salvar otra vida, porque para decidirlo en un momento crucial es necesario que la idea se haya considerado y discutido previamente, con reflexión y calma.']


In [30]:
def mergePages(firstPage, secondPage):
    unformatedPage = firstPage + secondPage
    skipPage = False
    finalPage = list()
    
    for i, block in enumerate(unformatedPage):
        paragraph = ''
        lastChar = block[-1]
        if (not skipPage):            
            if (lastChar == '.'):
                paragraph = block
            else:
                nexBlock = unformatedPage[i+1] if i < len(unformatedPage) - 1 else ''
                paragraph = block + ' ' + nexBlock
                skipPage = True 
            finalPage.append(paragraph)
        else:
            skipPage = False
    
    return finalPage

In [31]:
def transformDocument(listOfUTexts):
    cleanListOfText = list()

    for textTuple in listOfUTexts:
        questionaryFound = False
        firstPage, secondPage = textTuple
        arrParagrahps = []
        text = Text()
        title, cleanFirstPage, questionaryFound = cleanPage(firstPage, questionaryFound, True)
        
        if (cleanFirstPage):
            cleanSecondPage, questionaryFound = cleanPage(secondPage, questionaryFound)
            arrParagrahps = mergePages(cleanFirstPage, cleanSecondPage)
            text.setTitle(title)
            text.setUnformatedParagraphs(arrParagrahps)
            text.formatParagraphs()
            cleanListOfText.append(text)

    return cleanListOfText

In [32]:
results = transformDocument(listOfTexts)

In [33]:
results

In [34]:
results[0].getTitle()

'El acto solidario de la donación de órganos1'

In [35]:
for index, resultT in enumerate(results):
    resultT.save(database)
    print(index, resultT.toString())
    print('========================================================================')

0 Si bien los trasplantes se han convertido en una práctica habitual aún persisten fuertes temores en la población para donar órganos lograr su superación es la clave para aumentar el número de los donadores solidarios que hacen falta para salvar miles de vidas.
Es preciso entonces que se aclaren algunas dudas para que las personas pierdan el miedo a donar. Primero que lo complicado de los procedimientos de extirpación y trasplantación en el que intervienen varios equipos médicos altamente especializados vuelve muy difícil la existencia de mafias. Segundo que la necesaria compatibilidad afinidad de grupo sanguíneo entre donante y receptor dificulta la posibilidad de muertes a pedido.
La última cuestión es la más compleja en la actualidad aunque alguien haya manifestado expresamente su voluntad de donar es a la familia a la que se consulta en el momento en que la donación puede efectuarse. Como se entiende tal consulta llega en un momento difícil y poco propicio para las reflexiones pro

7 El sistema para nombrar a estos fenómenos atmosféricos busca facilitar que la población los identifique.
El huracán Patricia es el más poderoso registrado en el Pacífico mexicano y tiene vientos máximos cercanos a los 325 kilómetros por hora. Es catalogado como la tormenta más fuerte documentada hasta el momento.
Pero cómo deciden ponerles nombres a estos fenómenos atmosféricos Cada año se prepara una lista con los nombres que recibirán los huracanes que vayan ocurriendo a lo largo de la temporada.
Esta lista que se repite cada seis años incluye un nombre por cada letra del alfabeto y se alternan nombres masculinos y femeninos. Se buscan nombres fáciles de recordar para optimizar la comunicación escrita y que de esta forma sea más sencillo alertar a la población.
Al inicio se nombraba a las tormentas de forma arbitraria. Sin embargo a mediados del siglo XIX se decidió identificar a las tormentas con nombres de mujer. En 1979 comenzaron a incluirse también nombres de hombres para las 

15 Procure instalar un detector de humo.
Chequee constantemente llaves uniones y cilindros que contengan cualquier tipo de gas inflamable.
Si hay humo agáchese y gatee.
Siga las instrucciones que le indiquen los cuerpos de socorro.
Aléjese del incidente y permita que los cuerpos de socorro concluyan con su labor.
No sobrecargue las instalaciones eléctricas.
Si su ropa arde no corra deténgase agáchese y ruede en el piso para apagar el fuego.
Si hay heridos pida auxilio a los cuerpos de socorro.

16 Se cree que fue Hera la esposa de Zeus el dios de dioses la que dio origen a la Vía Láctea nuestra galaxia.
Zeus era muy aventurero y le gustaba mucho tener diferentes mujeres por lo que nunca le guardó fidelidad a su esposa. En una de estas aventuras Zeus se unió con Alcmena en ausencia de su esposo. El dios se hizo pasar por el ausente y decidió estar con ella en una noche que durase más tiempo del normal para ello ordenó al sol que no saliera cuando tenía que hacerlo.
Después el esposo de 

21 Ingredientes  ½ kg de mondongo cocido y cortado en tiras.
 ½ kg de papas fritas en tiras.
 ½ taza de alverjitas cocidas.
 ¼ de taza de aceite.
 1 cebolla cortada finamente.
 1 tomate grande pelado y picado.
 2 cucharadas soperas de pasta de tomate.
 1 cucharadita de pimiento molido.
 Hongos.
 Laurel.
 Sal.
Preparación  Para empezar a preparar este delicioso plato de mondonguito a la italiana con arroz blanco pon a hervir el mondongo con agua y sal. Luego cuando notes que ya pasó un tiempo considerable escúrrelo y córtalo en tiras.
 Por otro lado prepara el aderezo. Fríe la cebolla el tomate la pasta de tomate el pimiento los hongos y el laurel. Déjalo cocinar con un poco de agua y después échale su punto de sal.
 Cuando el aderezo hierva agrega el mondongo las alverjitas y las zanahorias. Ten papas fritas listas agrégalas y revuelve todo para que adquieran los sabores. Agrega el perejil y listo.

22 Una corazonada nos dice que el origen del anticucho se pierde en la inmensidad del t

30 Hay muchos caminos que conducen a Machu Picchu pero ninguno como el Camino Inca el más popular entre los viajeros y la vía peatonal más famosa del continente americano. Desde Cusco 43 kilómetros de una ruta entre bosques y densas nieblas escalones de piedra milenarios y vistas majestuosas.
Al final está la recompensa la famosa Puerta del Sol y sus impresionantes vistas de las ruinas de Machu Picchu.
Hacer el Camino Inca es un rito para el viajero y una aventura única. El trayecto más largo se inicia en Piscacucho km 82 de la línea férrea hacia Machu Picchu. Atraviesa diferentes ecosistemas colosales sitios arqueológicos y parajes ricos en flora y fauna hasta llegar a la ciudadela de Machu Picchu.
Es importante escoger la mejor época. Se organizan grupos todo el año excepto en febrero cuando llueve a cántaros y el Camino Inca se cierra por mantenimiento pero los mejores meses más fríos y secos son junio julio y agosto.
Para realizar la excursión es indispensable contactar con una age

38 Tener conciencia ecológica es entender que somos dependientes de la naturaleza y responsables de su estado de conservación. Ignorar esta verdad equivale a autodestruirnos porque al degradar el ambiente estamos empeorando nuestra calidad de vida y poniendo en peligro el futuro de nuestros descendientes.
Al contrario de lo que se podría pensar desarrollo económico y conciencia ecológica no tienen por qué ser antónimos. Se puede producir de manera sustentable fomentar la generación de energías limpias reducir y reciclar la basura y reutilizar un gran número de materiales que producirán nuevos productos útiles para el día a día del hombre moderno.
Es cierto que la conciencia ecológica debe comenzar en el seno del hogar pero debe extenderse a todos los ámbitos de nuestra existencia porque simplemente todas nuestras acciones inciden de manera positiva o negativa sobre la naturaleza.
Una actitud positiva por nuestro ambiente Mientras ellos te prometen la luna nosotros te garantizamos la Ti

41 El león siempre conocido por ser el rey de la selva es muy sociable es el único felino que vive en manadas usualmente compuestas por hasta 15 hembras de 1 a 3 machos y muchos cachorros todos ellos menores de tres años.
Llega a medir entre 14 y 230 metros sin incluir la cola la cual puede medir de 68 a 100 cm. De forma silvestre viven entre 10 y 16 años mientras que en cautiverio pueden llegar a vivir entre 20 y 25 años.
Los machos se diferencian fácilmente de las hembras ya que tienen una melena abundante además de ser más robustos. Algunos machos tienen la melena de color negra pero la mayoría la tiene de color más claro aunque más oscura que el color del pelaje del cuerpo que es de color dorado. Los machos pesan un 45  más que las hembras llegando a alcanzar los 225 kg.
La vista del león es muy buena tiene una visión nocturna seis veces mejor que la de los humanos. Sus oídos son tan sensibles que le permiten escuchar hasta la presa más pequeña un ratón. Además posee una mandíbula 